In [1]:
import pandas as pd
import numpy as np
import json
import torch

In [2]:
!pip install --user git+https://github.com/maciejkula/spotlight.git@master#egg=spotlight

In [3]:
!pip install --user --upgrade pip

Requirement already up-to-date: pip in /users/msc/2413917c/.local/lib/python3.6/site-packages (19.2.1)


In [2]:
df = pd.read_csv('amazon_reviews_us_Electronics_v1_00.tsv.gz', delimiter='\t',error_bad_lines=False)
df.head()

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [103]:
df1 = df.copy()
df1.count()

marketplace          3091024
customer_id          3091024
review_id            3091024
product_id           3091024
product_parent       3091024
product_title        3091020
product_category     3091024
star_rating          3091024
helpful_votes        3091024
total_votes          3091024
vine                 3091024
verified_purchase    3091024
review_headline      3090993
review_body          3090936
review_date          3091000
dtype: int64

In [104]:
df1 = df1[df1.review_date.notnull()]
df1 = df1[df1.product_title.notnull()]
df1 = df1[df1.product_id.notnull()]
df1 = df1[df1.customer_id.notnull()]
df1.count()

marketplace          3090996
customer_id          3090996
review_id            3090996
product_id           3090996
product_parent       3090996
product_title        3090996
product_category     3090996
star_rating          3090996
helpful_votes        3090996
total_votes          3090996
vine                 3090996
verified_purchase    3090996
review_headline      3090965
review_body          3090908
review_date          3090996
dtype: int64

In [105]:
df1['review_date'] = pd.to_datetime(df1['review_date']).dt.strftime("%Y%m%d")

In [470]:
# from spotlight.sampling import sample_items
# df1 = df1.sample(1000000, random_state=1)
len(np.where(df1[['customer_id']].groupby(df['customer_id']).count()>2)[0])

183711

In [106]:
# mean of goup by item
print(df1[['product_id']].groupby(df1['product_id']).count().mean())
# mean of goup by userfrom spotlight.sampling import sample_items
print(df1[['customer_id']].groupby(df['customer_id']).count().mean())
print(df1[['star_rating']].astype('int').mean())

product_id    16.638743
dtype: float64
customer_id    1.435831
dtype: float64
star_rating    4.035421
dtype: float64


In [451]:
df1[['customer_id']].groupby(df['customer_id']).count().max()

customer_id    11
dtype: int64

In [260]:
df1.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [42]:
from spotlight.sampling import sample_items
df1 = df1.sample(1000000, random_state=1)

In [107]:
min_r = 1850
num_review = df1[['product_id']].groupby(df1['product_id']).count()
print(len(np.where(num_review>min_r)[0]))
print(len(num_review))
item = num_review.index[np.where(df1[['product_id']].groupby(df1['product_id']).count()>min_r)[0]]
len(item)
# group = [df1[df1['product_id']==item[i]] for i in range(len(item))]

100
185771


100

In [108]:
start = time.time()
df2 =  pd.DataFrame( columns = df1.columns) 
for i in range(len(item)):
    df2 = df2.append(df1[df1['product_id']==item[i]])
end = time.time()
diff = end-start
print('Data processing cost %d seconds' % (diff))

Data processing cost 84 seconds


In [109]:
print(100*(df2[['star_rating']].groupby(df2['star_rating']).count()/len(df2)))
print(df2[['star_rating']].groupby(df2['star_rating']).count())
np.min(df2[['star_rating']].groupby(df2['star_rating']).count())

             star_rating
star_rating             
1               6.766978
2               4.261885
3               6.266190
4              16.068151
5              66.636796
             star_rating
star_rating             
1                  23485
2                  14791
3                  21747
4                  55765
5                 231265


star_rating    14791
dtype: int64

In [110]:
from spotlight.sampling import sample_items
df_g1 =  pd.DataFrame( columns = df1.columns) 
num_sample = np.min(df2[['star_rating']].groupby(df2['star_rating']).count())
for i in range(len(df2['star_rating'].unique())):
    df_g1 =  df_g1.append(df2[df2['star_rating']==i+1].sample(10000, random_state=1))
# df2

In [111]:
df_g1[['star_rating']].groupby(df_g1['star_rating']).count()

,star_rating
star_rating,
1,10000
2,10000
3,10000
4,10000
5,10000


In [112]:
df1 = df_g1

In [113]:
from collections import defaultdict
from itertools import count

uid_map = defaultdict(count().__next__)
iid_map = defaultdict(count().__next__)
uids = np.array([uid_map[uid] for uid in df1["customer_id"].values ], dtype=np.int32)
iids = np.array([iid_map[iid] for iid in df1["product_id"].values], dtype=np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}

ratings = df1["star_rating"].values.astype(np.float32)
timestamps = df1["review_date"].values.astype(np.object)

print("userId %d got uid %d" % (41409413, uid_map[41409413]))
print("itemId %s got iid %s" % ('B00428R89M', iid_map['B00428R89M']))

userId 41409413 got uid 49273
itemId B00428R89M got iid 100


In [114]:
ratings

array([1., 1., 1., ..., 5., 5., 5.], dtype=float32)

In [115]:
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split


dataset = Interactions(user_ids=uids,
                        item_ids=iids,
                        ratings=ratings,
                        timestamps = timestamps)

train, test = random_train_test_split(dataset, random_state=np.random.seed(42))

In [125]:
print(train)
print(test)

<Interactions dataset (49273 users x 100 items x 40000 interactions)>
<Interactions dataset (49273 users x 100 items x 10000 interactions)>


In [117]:
df1

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
358044,US,17766886,R12F1BW2ZWEO8F,B00E0GRRR4,318818065,66 AUDIO - BTS+ SPORT - Bluetooth Wireless Hea...,Electronics,1,0,2,N,Y,"Too small, uncomfortable, average sound",The band on these headphones is much too small...,20150509
2758498,US,14204446,R2F00K4CWKGY2K,B002MAPS6W,808084969,Clip Plus 4 GB MP3 Player (Black),Electronics,1,1,10,N,Y,PIECE OF JUNK!!!!!!!!!!!,GOT THIS PLAYER FOR X-MAS. 2 WEEKS LATER THE C...,20100217
2835347,US,52287732,R16D5YMHAEIAZZ,B000C1Z0HA,181905409,Next Generation Remote Control Extender,Electronics,1,2,2,N,N,Didn't Work!,While this product makes big promises; it does...,20090429
240139,US,29464499,R11Z9WJAI15C2N,B0007MXZB2,170879963,Terk HDTVAZ Amplified Indoor HDTV Antenna,Electronics,1,0,0,N,Y,One Star,It didn't work for me. I don't know why.,20150618
123717,US,36088711,R8BVEEQ4MOQYX,B00870ZHCQ,902415312,AmazonBasics High-Speed HDMI Cable - 3 Feet (0...,Electronics,1,0,0,N,Y,Bad cable--had to send back,Bad cable--had to send back,20150725
703369,US,44369143,R6HD9DG89JAEH,B0030AZ44O,211712843,ZAZUS002577,Electronics,1,0,0,N,Y,Very disappointing.,"Not only did this not work as advertised, it d...",20150128
1142445,US,16201363,R1AWNY9ILSRQ6M,B0006B486K,440305749,Koss KSC75 Portable Stereophone Headphones,Electronics,1,0,0,N,Y,Poor Quality!,In less than 2 months the wire in the right ea...,20140904
916048,US,12855360,RPGQN8YXWK3NW,B008KVUAGU,781032079,Mediasonic Homeworx HW110AN Super Thin Indoor ...,Electronics,1,0,0,N,Y,On the internet it says I am supposed to get ...,On the internet it says I am supposed to get c...,20141201
1066982,US,2060104,R34BLQKNQTL001,B0052SCU8U,572574607,AmazonBasics High Speed HDMI Cable,Electronics,1,0,1,N,Y,One Star,It's not grat,20141002
453375,US,5495339,RQCTS6OMIFWEK,B000IJY8DS,654766735,"High Speed HDMI Cable Supports Ethernet, 3D an...",Electronics,1,0,0,N,Y,One Star,It's was bad,20150407


In [118]:
#map userId to the internal uid value
userId =48540322

uid = uid_map.get(userId)

#see which ratings are for this user. Use this to filter the item and ratings arrays
print(train.item_ids[train.user_ids == uid])
print(train.ratings[train.user_ids == uid])

[27]
[5.]


In [119]:
train.ratings

array([4., 2., 2., ..., 2., 2., 1.], dtype=float32)

Sample_G1

In [126]:
from spotlight.factorization.explicit import ExplicitFactorizationModel
import time  

emodel = ExplicitFactorizationModel(n_iter= 300,
                                    embedding_dim=64)
current = time.time()

emodel.fit(train, verbose=True)

end = time.time()
diff = end - current
print("Training took %d seconds "% (diff))

Epoch 0: loss 7.806796341185358
Epoch 1: loss 2.022781139726092
Epoch 2: loss 0.6610548025483538
Epoch 3: loss 0.2969392208726543
Epoch 4: loss 0.1992007658170287
Epoch 5: loss 0.1424425827090148
Epoch 6: loss 0.10569257844405569
Epoch 7: loss 0.08861944499384065
Epoch 8: loss 0.07792187346869213
Epoch 9: loss 0.07348743546160923
Epoch 10: loss 0.07170412976555764
Epoch 11: loss 0.07156011061209022
Epoch 12: loss 0.07324569494386388
Epoch 13: loss 0.07386452161297677
Epoch 14: loss 0.07325337929235902
Epoch 15: loss 0.07416254049463636
Epoch 16: loss 0.07551540249282387
Epoch 17: loss 0.07590946101933528
Epoch 18: loss 0.07800891183933635
Epoch 19: loss 0.07647202600529239
Epoch 20: loss 0.07430210358397976
Epoch 21: loss 0.07331226566794571
Epoch 22: loss 0.07272510918652177
Epoch 23: loss 0.07150487207872852
Epoch 24: loss 0.06834878112859787
Epoch 25: loss 0.06850228283056028
Epoch 26: loss 0.0689685718411473
Epoch 27: loss 0.0677774439856505
Epoch 28: loss 0.0664784680743506
Epoch 

Epoch 229: loss 0.03710195992830073
Epoch 230: loss 0.034347283458159225
Epoch 231: loss 0.03659579590864622
Epoch 232: loss 0.035342485329527766
Epoch 233: loss 0.03839286091695925
Epoch 234: loss 0.0363969151642482
Epoch 235: loss 0.03670641507028015
Epoch 236: loss 0.036401339397308934
Epoch 237: loss 0.04007569634041209
Epoch 238: loss 0.03490271580636881
Epoch 239: loss 0.035307481790044504
Epoch 240: loss 0.03550037133038803
Epoch 241: loss 0.03756867274404711
Epoch 242: loss 0.034947230605183136
Epoch 243: loss 0.03295478074081764
Epoch 244: loss 0.0349938413662136
Epoch 245: loss 0.039222205983130795
Epoch 246: loss 0.038268744269279155
Epoch 247: loss 0.03824756799894533
Epoch 248: loss 0.038476576696440674
Epoch 249: loss 0.03574594224144699
Epoch 250: loss 0.03657977080126856
Epoch 251: loss 0.03551472116048169
Epoch 252: loss 0.03680010423491335
Epoch 253: loss 0.03586602820807202
Epoch 254: loss 0.03574573498954819
Epoch 255: loss 0.036140198884591176
Epoch 256: loss 0.036

iteration: 400, dim: 64, (49274 users x 100 items) Training took 3011 seconds

In [124]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.198, test RMSE 1.353


iteration: 300, dim: 64, (49274 users x 100 items) Training took  seconds

In [121]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.198, test RMSE 1.350


iteration: 400, dim: 64, (49274 users x 101 items) Training took 2897 seconds 

In [99]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.194, test RMSE 1.358


iteration: 200, dim: 64, (49274 users x 101 items)

In [97]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.206, test RMSE 1.356


iteration: 100, dim: 64, (49274 users x 101 items)

In [95]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.227, test RMSE 1.357


iteration: 400, dim: 64, (26219 users x 119 items), 382seconds

In [63]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.161, test RMSE 1.346


iteration: 300, dim: 64, (26219 users x 119 items), 382seconds

In [61]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.168, test RMSE 1.343


iteration: 200, dim: 64, (26219 users x 119 items), 382seconds

In [59]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.181, test RMSE 1.347


iteration: 100, dim: 64, (26219 users x 119 items)

In [57]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.218, test RMSE 1.415


iteration: 300, dim: 64, (97381 users x 248 items) Training took 8534 seconds 

In [29]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.360, test RMSE 1.351


iteration: 200, dim: 64, (97381 users x 248 items) 

In [27]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.358, test RMSE 1.348


iteration: 100, dim: 64, (97381 users x 248 items) Training took 2273 seconds 

In [25]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.388, test RMSE 1.342


iteration: 10

In [317]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.207, test RMSE 2.302


iteration: 20

In [320]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.225, test RMSE 2.175


iteration: 100

In [322]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.153, test RMSE 1.465


iteration: 100, dim: 64, 117493 users x 248 items, Training took 3599 seconds

In [442]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.409, test RMSE 1.347


iteration: 200, dim: 64, 117493 users x 248 items, Training took 8147 seconds 

In [446]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.395, test RMSE 1.354



iteration: 200

In [324]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.120, test RMSE 1.342


iteration= 200
dim = 64

In [326]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.178, test RMSE 1.333


iteration= 300 dim = 64

In [328]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.156, test RMSE 1.326


iteration= 400 dim = 64

In [335]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.161, test RMSE 1.325


iteration= 500 dim = 64

In [333]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.149, test RMSE 1.329


iteration= 1000 dim = 64

In [331]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 0.153, test RMSE 1.341


1000000

In [11]:
from spotlight.factorization.explicit import ExplicitFactorizationModel
import time  

emodel = ExplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32, #this is Spotlight default
                                    use_cuda=False)
current = time.time()

emodel.fit(train, verbose=True)

end = time.time()
diff = end - current
print("Training took %d seconds "% (diff))

Epoch 0: loss 9.63062810623169
Epoch 1: loss 2.6582441970062254
Epoch 2: loss 2.8112086178207396
Epoch 3: loss 2.5809681952667236
Epoch 4: loss 2.182701927986145
Epoch 5: loss 2.7678971632385254
Epoch 6: loss 2.1409205086898804
Epoch 7: loss 2.632769370422363
Epoch 8: loss 2.182251567802429
Epoch 9: loss 2.551185591506958
Training took 21352 seconds 


In [65]:
from spotlight.factorization.explicit import ExplicitFactorizationModel
import time  

emodel = ExplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32, #this is Spotlight default
                                    use_cuda=False)
current = time.time()

emodel.fit(train, verbose=True)

end = time.time()
diff = end - current
print("Training took %d seconds "% (diff))

Epoch 0: loss 9.632991309051514
Epoch 1: loss 2.669210560951233
Epoch 2: loss 2.8042855801773072
Epoch 3: loss 2.5828313162994383
Epoch 4: loss 2.180853924827576
Epoch 5: loss 2.7598054175567626
Epoch 6: loss 2.1496787886810305
Epoch 7: loss 2.634869267463684
Epoch 8: loss 2.185838869934082
Epoch 9: loss 2.545376735458374
Training took 31130 seconds 


In [42]:
from spotlight.factorization.explicit import ExplicitFactorizationModel
import time  

emodel = ExplicitFactorizationModel(n_iter=10,
                                    embedding_dim=32, #this is Spotlight default
                                    use_cuda=False)
current = time.time()

emodel.fit(train, verbose=True)

end = time.time()
diff = end - current
print("Training took %d seconds "% (diff))

Epoch 0: loss 11.581687960306803
Epoch 1: loss 3.2796892013549805
Epoch 2: loss 2.4529409646987914
Epoch 3: loss 2.5543935860951743
Epoch 4: loss 1.963225773747762
Epoch 5: loss 2.3609276292800905
Epoch 6: loss 1.89075053024292
Epoch 7: loss 2.0199421435038247
Epoch 8: loss 1.8515831254959108
Epoch 9: loss 2.033038276672363
Training took 8091 seconds 


In [443]:
import torch
torch.save(emodel, 'explicit_model_1000_iter')
emodel = torch.load("explicit_model_1000_iter")

Sample 300,000 users (10 iterations)

In [54]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 1.305, test RMSE 2.634


Sample 600,000 users (10 iterations)

In [44]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 1.572, test RMSE 2.211


Sample 1,000,000 users (10 iterations)

In [13]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 1.815, test RMSE 2.010


Original dataset (3 iterations)

In [10]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(emodel, train)
test_rmse = rmse_score(emodel, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

Train RMSE 2.250, test RMSE 1.901


In [349]:
df1


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
454751,US,48837738,R2MQNBQK30G1JO,B00APPDX86,708525197,Mohu Leaf 50 TV Antenna Amplified 50 Mile Rang...,Electronics,1,0,1,N,Y,I had to jimmy it and pretty much remake the c...,Right when I received it it was broken. I had ...,20150407
2202994,US,48089950,R3VWHBWOERQEH1,B0012S4APK,845709830,"Cheetah APTMM2B TV Wall Mount for 20-75"" TVs u...",Electronics,1,0,1,N,Y,Box had two left brackets and missing the righ...,Box had two left brackets and missing the righ...,20121229
194637,US,878932,R3G8PR64HA8CGS,B008KVUAGU,781032079,Mediasonic Homeworx HW110AN Super Thin Indoor ...,Electronics,1,0,1,N,Y,One Star,It works for a few moments and then starts ski...,20150703
1760075,US,10815490,R2YO9VCNWL5AOJ,B0049S6ZUS,452648296,Kinivo HDMI Switch 301BN Premium 3 Port Wirele...,Electronics,1,0,0,N,Y,it doesn't work,"This device sucks<br />i have the 3 port one, ...",20131021
496214,US,15281328,R9KEK22FF6IAC,B003CJTR82,41165686,Philips SHS3200WT/37 Flexible Earhook Headphon...,Electronics,1,0,0,N,Y,Got shocked the first time I used,You get what u pay for. First time I put these...,20150325
2755311,US,20822903,R3HCQQLTG17SCR,B0019EHU8G,443415697,Mediabridge ULTRA Series HDMI Cable (3 Foot) -...,Electronics,1,0,1,N,Y,Poor or No signal to 1080p TV,I ordered this cable (Mediabridge 6ft High Spe...,20100227
479805,US,17201864,R347XMY58XGAPL,B000IG66VS,742454107,JLab JBuds Hi-Fi Noise-Reducing Ear Buds (Purple),Electronics,1,1,1,N,Y,"CHANGED PRODUCT! Once Great, Now Garbage!",I bought two sets of these ear buds back in 20...,20150330
1067657,US,13652670,R3QWV3JR606B60,B003L1ZYYM,617978254,AmazonBasics High-Speed HDMI Cable - 6.5 Feet ...,Electronics,1,0,0,N,Y,"do not buy it, buy it on bestbuy",Not satisfied. I didnt want to pay 20usd on a ...,20141002
1348419,US,16664323,R1RRUYU37G5WUD,B000HKGK8Y,544321119,"RCA ANT111Z Durable FM Antenna, Rabbit Ears",Electronics,1,1,1,N,Y,Pointless,Did not work. I live in an appartment building...,20140607
603090,US,13231023,R1WY8LMMCJZY2A,B003L1ZYYM,617978254,AmazonBasics High-Speed HDMI Cable - 6.5 Feet ...,Electronics,1,0,5,N,Y,One Star,Didn't make a difference with the clarity of t...,20150224


In [379]:
testItemId = test.item_ids[np.where(test.user_ids == uid_map.get(48837738))]
testItemId
# predicted = emodel.predict( np.array([]), item_ids=np.array([0, testItemId]))
# predicted

array([12])

In [465]:
allpreds = emodel.predict( np.array(2), item_ids=None)
print(allpreds)

[-2.5849414  -0.05704141  0.6923404   0.630389    0.4216411   0.77651465
 -0.35550022  0.05244517  0.29165506 -0.04372597  0.5194247   1.1483679
 -0.3481835   0.1652224  -0.4910791   0.4941306  -0.6293678   0.09735751
  1.2529736   0.16102672  0.02662706  0.81928253  0.06198502 -0.9202218
  2.1857686   0.34072304  0.21218586  0.16232705 -0.10655069  0.16899061
  0.13938975  0.19854069 -0.05350494  0.57360005 -0.6505492   0.14334846
  0.7936704   0.5692754   0.60989666 -0.84435487  0.11110353 -0.3047049
  0.5299535  -1.1451674   0.3905201  -0.08318806  0.15153146  0.4680636
  0.44803047  0.21750832  0.5054953   0.6203551   0.27098894 -0.3698368
  0.8096143   0.340106    0.2577076  -0.06992269  0.22134113  0.17660856
  0.5042982  -0.19443417  0.02915382  0.01496983  0.25890565  0.10359144
  0.83954644  0.4262178   0.27585125 -0.2463913   0.41659307  0.5618212
  0.35779238  0.4094634   0.4048376  -0.58132577  0.10951614  0.08227873
  0.15901041 -0.12259531 -0.37439418  0.20202565 -0.38048

In [461]:
allpreds = emodel.predict( uid_map.get(48837738), item_ids=None)
print(allpreds)
print(allpreds.size)
np.mean(allpreds)

[-1.4629111   1.9080439   1.9983668   1.5273505   1.3923751   1.0927353
  1.4215703   1.3515227   1.4389663   0.8781812   2.0645044   1.777585
  0.23742485  0.8917223   1.3406351   1.5793679   0.32249367  0.7069099
  2.1412382   1.3329242   1.7169747   1.9426236   2.0124922   0.96638846
  1.2608156   1.4242908   2.2702515   1.5399252   1.868582    1.2008579
  1.2030488   2.250675    1.2588036   2.0802214   1.1767825   1.1045693
  2.2530088   1.6485811   0.96308374  0.75110877  1.9986217   1.0264108
  1.9917202   0.31299663  1.762838    0.8665824   0.709448    1.9063873
  2.045577    1.0898343   1.4262605   1.501258    1.9803803   1.4306887
  1.3415357   1.7920454   1.0001932   0.9520649   1.2764907   1.458131
  0.67144084  1.0174615   0.31666446  1.9972099   1.4567562   1.1016114
  2.1280284   1.7857581   1.4278092   1.4760685   1.7400224   1.8285306
  1.2126846   1.8998244   1.0352267   0.6664406   1.2431722   0.8780227
  1.8736981   1.5063684   0.67682624  1.7291045   1.0540812   1.5

1.590533

In [45]:
emodel._net.item_embeddings.weight[0]

tensor([-0.0361, -0.0052,  0.0239, -0.0377, -0.0368, -0.0190, -0.0354, -0.0023,
        -0.0599, -0.0101,  0.0248,  0.0215, -0.0485,  0.0142, -0.0054, -0.0397,
        -0.0518, -0.0177, -0.0150,  0.0087,  0.0010,  0.0271,  0.0052,  0.0026,
         0.0220,  0.0069, -0.0058,  0.0249,  0.0101,  0.0108, -0.0565, -0.0364],
       grad_fn=<SelectBackward>)

In [57]:
import numpy as np
import pandas as pd
import scipy

In [66]:
a = 1
b = 5
data_norm_to_a_b = [(number - a)/(b - a) for number in allpreds]
data_norm_to_a_b

[-0.17329905182123184,
 0.6344529390335083,
 0.544802188873291,
 0.34530484676361084,
 0.4301719069480896,
 0.7183942794799805,
 0.9332412481307983,
 1.0602376461029053,
 0.6878787279129028,
 0.824652910232544,
 0.4738435745239258,
 0.4807004928588867,
 0.9311654567718506,
 -0.15809591859579086,
 1.44987154006958,
 0.7133413553237915,
 0.8264908790588379,
 0.9110202789306641,
 1.1013188362121582,
 0.7890945672988892,
 1.0860595703125,
 0.810055136680603,
 0.6506012678146362,
 0.2107868790626526,
 0.790966272354126,
 0.4920668601989746,
 0.9070143699645996,
 -0.026108741760253906,
 0.646919310092926,
 0.6612879037857056,
 0.925434947013855,
 0.48245906829833984,
 0.47854214906692505,
 0.37077683210372925,
 0.5691391825675964,
 0.3943747282028198,
 0.1878662109375,
 0.591887354850769,
 0.7704563140869141,
 -0.10019004344940186,
 0.9080613851547241,
 0.6387462615966797,
 0.8443186283111572,
 0.6661134362220764,
 0.8187744617462158,
 0.29959768056869507,
 0.614789605140686,
 0.530382513999